# L-BFGS
## Wolfe
$$
\begin{cases}
f(x_k + {\lambda_k}{d_k}) &\leq f(x_k) + \rho {\lambda_k}{d_k^T}{g_k}\\
{d_k^T}g(x_k + {\lambda_k}{d_k}) &\geq \sigma{d_k^T}{g_k}
\end{cases}
$$
其中，$\rho \in (0, 0.5)$, $\sigma \in (\rho, 1)$


In [ ]:
class L_BFGS(object):
    self.lambda_ = 0.55
    def __init__(self, rho, sigma, epsilon = 0.1, m = 10000) -> None:
        if rho >= 0.5 or rho <= 0:
            raise(ValueError("rho need in (0, 0.5)"))
        if sigma <= rho or sigma >= 1:
            raise(ValueError("sigma need in (rho, 1)"))
        self.rho = rho
        self.sigma = sigma
        self.epsilon = epsilon
        # 内存值
        self.m = m

    
    def wolfe_search(self, fun, gfun, xk, gk, dk):
        '''
        fun：原始函数
        gfun：梯度函数
        xk：kth x值
        gk：kth 梯度值
        dk：kth D
        '''
        m = 0
        mk = 0
        while m < 20: # 用Wolfe条件搜索求步长
            new_gk = gfun(xk + self.lambda_**m*dk)
            if fun(xk+self.lambda_**m*dk) < fun(xk)+ self.rho*self.lambda_**m*np.dot(dk.T, gk) and np.dot(dk.T, new_gk) >=  self.sigma*np.dot(dk.T,gk):
                mk = m
                break
            m += 1
        return self.lambda_ ** mk
    def calc_Dkgk(self, k, Y, S, gk, D0):
        '''
        Dk.gk快速算法
        '''
        # step1: init
        delta = 0
        if k <= self.m:
            delta = 0
        else:
            delta = k - self.m

        length = 0
        if k <= self.m:
            length = k
        else:
            length = self.m
        
        q_s = [0] * (length+1)
        q_s[length] = gk

        alpha = [0] * (length+1)
        # step2: backward
        for i in range(length-1, -1, -1):
            j = i + delta
            alpha[i] = np.linalg.inv(np.dot(Y[j].T, S[j])).dot(S[j].T) * q_s[i+1]
            q_s[i] = q_s[i+1] - alpha[i]*Y[j]

        # step3: foreward
        r_s = [0] * (length+1)
        r_s[0] = D0*q_s[0]
        beta = [0] * (length+1)
        for i in range(length):
            j = i + delta
            beta[i] = np.linalg.inv(np.dot(Y[j].T, S[j])).dot(Y[j].T)*r_s[i]
            r_s[i+1] = r_s[i] + (alpha[i] - beta[i]) * S[j]
        return r_s[length + 1]
    def run(self, fun, gfun, X):
        # 初始化D0, k
        N = X.shape[0]
        D0, I = np.mat(np.identity(N)), np.mat(np.identity(N))
        k = 0

        x, S, Y = X, [], []
        gk = gfun(x)
        while True and k < 1000:
            if k == 0:
                Dk = D0
                dk = -Dk*gk
            else:
                dk = self.calc_Dkgk(k, Y, S, gk, D0)
            
            S.append(self.wolfe_search(fun, gfun, x, gk, dk) * x)
            x = x + S[k]
            gk_plus = gfun(x)
            if np.linalg.norm(gk_plus) < self.epsilon:
                break
            Y.append(gk_plus - gk)
            gk = gk_plus
            tmp = S[k].dot(Y[k].T).dot(np.linalg.inv(np.dot(Y[k].T, S[k])))
            Dk = (I - tmp).dot(Dk).dot(I - tmp) + tmp
            k = k + 1
        return x

